# Runnable

A Runnable is a LangChain concept representing an object with `invoke`, `stream` and `batch` methods. Runnables can sometimes can be combined.

## Invoke

The `invoke` method triggers the request to LLM.

In most cases, it returns an `AIMessage`, but in some special cases, it can return some special output. For example, structured output langchain object return `dict` or `Pydantic.BaseModel`.

---

The following cell shows the kind of object `langchain_core.language_models.BaseChatModel` heir returns.

In [7]:
model.invoke("Hello world")

AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2026-01-06T14:39:17.313641499Z', 'done': True, 'done_reason': 'stop', 'total_duration': 23004167206, 'load_duration': 22818871591, 'prompt_eval_count': 12, 'prompt_eval_duration': 27074996, 'eval_count': 10, 'eval_duration': 141396789, 'model_name': 'llama3.1', 'model_provider': 'ollama'}, id='lc_run--fe6d8bde-d9c8-4e4c-a7f2-df3839f539a6-0', usage_metadata={'input_tokens': 12, 'output_tokens': 10, 'total_tokens': 22})

## Callbacks

You can modify the behaviour of your Runnable using a callback. Extend the `langchain_core.callbacks.BaseCallbackHandler` with special methods and specify that Runnable have to use the callback handler during the infvocation in the "callbacks" attribute of the `config`.

The following table shows some of the methods that can be defined in the callback handler.

|      Method      |           Triggered When...          |                  Useful For...                 |
| :--------------: | :----------------------------------: | :--------------------------------------------: |
|  on_chain_start  |      A Chain (or Agent) begins.      |              Logging user inputs.              |
|   on_llm_start   |  The LLM API is about to be called.  | Inspecting the exact prompt sent to the model. |
|   on_llm_end     |  The LLM API returned the result.    | Logging the information about model outputs.   |
| on_llm_new_token | A token streams in (streaming only). |      Real-time UI updates (typing effect).     |
|   on_tool_start  |    An Agent decides to use a tool.   |     Debugging which tools are being picked.    |
|  on_chain_error  |         An exception happens.        | Capturing stack traces or alerting developers. |

---

The following cell defines the `MyCustomTracer`, which extends the behaviour of the runnable associated with invoking LLMs.

In [10]:
from typing import Any, Dict, List
from langchain_core.callbacks import BaseCallbackHandler
from langchain_core.outputs import LLMResult


class MyCustomTracer(BaseCallbackHandler):
    def on_llm_start(
        self, serialized: Dict[str, Any], prompts: List[str], **kwargs: Any
    ) -> Any:
        print(f"\n[MY-TRACER] LLM Started with prompt: {prompts[0][:50]}...")

    def on_llm_end(self, response: LLMResult, **kwargs: Any) -> Any:
        content = response.generations[0][0].text
        print(f"[MY-TRACER] LLM Finished. Output: {content[:50]}...")

The following cell illustrates the invocation of the model with attached callback.

In [9]:
from langchain_ollama import ChatOllama
model = ChatOllama(model="llama3.1")
ans = model.invoke(
    "Hello, I'm Fedor",
    config={"callbacks": [MyCustomTracer()]}
)


[MY-TRACER] LLM Started with prompt: Human: Hello, I'm Fedor...
[MY-TRACER] LLM Finished. Output: Nice to meet you, Fedor! Is there something I can ...
